In [0]:
import cv2
import dlib
from keras.models import load_model
import numpy as np
from numpy import expand_dims
import math
from google.colab.patches import cv2_imshow

In [0]:
face_detector = dlib.cnn_face_detection_model_v1("/content/drive/My Drive/Datasets/model/mmod_human_face_detector.dat")
model = load_model('/content/drive/My Drive/Datasets/model/facenet_keras.h5')

In [0]:
def get_image_face(img):
    
    faces = face_detector(img, 1)
    crop_faces=[]
    for face in faces:
        crop_faces.append(img[face.rect.top():face.rect.bottom(),
                       face.rect.left():face.rect.right()])
    return crop_faces[0]

In [0]:
def generate_embeddings(face_pixels):
    face_pixels=cv2.resize(face_pixels,(160,160))
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [0]:
# Example image
img = cv2.imread("/content/drive/My Drive/Datasets/Dataset/robertdownejr/robertdowneyjr_31913316_225277851571397_8189716001621803008_n.jpg")
imgc= cv2.resize(img,(200,250))
cv2_imshow(imgc)

In [0]:
face=get_image_face(img)
cv2_imshow(face)

In [0]:
embd=generate_embeddings(face)
embd

In [0]:
#this embeddings extract face features
embd.shape

In [0]:
def generate_aclass(name,label):
  basedir="/content/drive/My Drive/Datasets/Dataset/"+name
  import os
  ls=[]
  for file in os.listdir(basedir):
    img = cv2.imread(basedir+"/"+file)
    face=get_image_face(img)
    embd=list(generate_embeddings(face))
    embd.append(label)
    ls.append(embd)
  return ls

In [0]:
bennidict=generate_aclass("bennidict",0)
chrishemsworth=generate_aclass("chrishemsworth",1)
robertdownejr=generate_aclass("robertdownejr",2)

In [0]:
FinalLs=bennidict + chrishemsworth + robertdownejr

In [0]:
FinalArr=np.array(FinalLs)

In [0]:
class_label = FinalArr[:, -1] # for last column
dataset = FinalArr[:, :-1] # for all but last column

In [0]:
dataset[0].shape

In [0]:
from sklearn.neighbors import KNeighborsClassifier
KNNmodel = KNeighborsClassifier(n_neighbors=3)

In [0]:
KNNmodel.fit(dataset,class_label)

## Testing

In [0]:
def test_model(img):
  face2=get_image_face(img)
  embd=generate_embeddings(face2)
  dc={0:"Benedict Cumberbatch", 1 : "Chris Hemsworth", 2: "Robert Downey, Jr."}
  return dc[int(KNNmodel.predict([embd]))]

In [0]:
img2 = cv2.imread("/content/drive/My Drive/Datasets/Test/chrisendgame.jpg")
cv2_imshow(img2)
print("\n\nNAME -->",test_model(img2))

In [0]:
img2 = cv2.imread("/content/drive/My Drive/Datasets/Test/robertDowneyjr.jpg")
cv2_imshow(img2)
print("\n\nNAME -->",test_model(img2))

In [0]:
img2 = cv2.imread("/content/drive/My Drive/Datasets/Test/benedict_cumberbatch.jpg")
cv2_imshow(img2)
print("\n\nNAME -->",test_model(img2))

Thank You